In [7]:
import numpy
import math
import pyparticleest.utils.kalman as kalman
import pyparticleest.interfaces as interfaces
import pyparticleest.paramest.paramest as param_est
import pyparticleest.paramest.interfaces as pestint
import matplotlib.pyplot as plt
from scipy.stats import bernoulli

In [8]:
def generate_dataset(steps, P0, Q):
    x = numpy.zeros((steps + 1,))
    y = numpy.zeros((steps + 1,))
    x[0] = numpy.random.multivariate_normal((0.0,), P0)
    y[0] = numpy.random.binomial(1,numpy.exp(x[0])/(1.0+numpy.exp(x)[0]))
    for k in range(0, steps):
        x[k + 1] = x[k] + numpy.random.multivariate_normal((0.0,), Q)
        y[k + 1] = bernoulli.rvs(numpy.exp(x[k+1])/(1.0 + numpy.exp(x[k+1])))

    return (x, y)

In [9]:
def wmean(logw, val):
    w = numpy.exp(logw)
    w = w / sum(w)
    return numpy.sum(w * val.ravel())


In [10]:
class Model(interfaces.FFBSiRS, interfaces.ParticleFiltering,
            pestint.ParamEstInterface, pestint.ParamEstBaseNumeric):
    """ x_{k+1} = x_k + v_k, v_k ~ N(0,Q),
        y_k = e_k, e_k ~ Bernoulli(\exp(x_{k})/(1+\exp(x_{k}))),
        x(0) ~ N(0,P0) """

    def __init__(self, P0, Q):
        self.P0 = numpy.copy(P0)
        self.Q = numpy.copy(Q)
        self.logxn_max = kalman.lognormpdf_scalar(numpy.zeros((1,)), self.Q)
        super(Model, self).__init__()

    def create_initial_estimate(self, N):
        return numpy.random.normal(0.0, numpy.sqrt(self.P0), (N,))

    def sample_process_noise(self, particles, u, t):
        """ Return process noise for input u """
        N = len(particles)
        return numpy.random.normal(0.0, numpy.sqrt(self.Q), (N,))

    def update(self, particles, u, noise, t):
        """ Update estimate using 'data' as input """
        particles[:] = particles + noise

    def measure(self, particles, y, t):
        """ Return the log-pdf value of the measurement """
        return bernoulli.logpmf(y,numpy.exp(particles)/(1.0 + numpy.exp(particles)))

    def logp_xnext(self, particles, next_part, u, t):
        """ Return the log-pdf value for the possible future state 'next' given input u """
        pn = particles
        return kalman.lognormpdf_scalar(pn.ravel() - next_part.ravel(), self.Q)
    def logp_xnext_max(self, particles, u, t):
        return self.logxn_max

    def sample_smooth(self, part, ptraj, anc, future_trajs, find, ut, yt, tt, cur_ind):
        """ Update ev. Rao-Blackwellized states conditioned on "next_part" """
        return part.reshape((-1, 1))

    def set_params(self, params):
        """ New set of parameters for which the integral approximation terms will be evaluated"""
        self.params = numpy.copy(params)
        self.Q = math.exp(params[0]) * numpy.eye(1)

    def eval_logp_x0(self, particles, t):
        """ Calculate gradient of a term of the I1 integral approximation
            as specified in [1].
            The gradient is an array where each element is the derivative with
            respect to the corresponding parameter"""
        return kalman.lognormpdf_scalar(particles, self.P0)

    def copy_ind(self, particles, new_ind=None):
        if (new_ind is not None):
            return numpy.copy(particles[new_ind])
        else:
            return numpy.copy(particles)

    def eval_logp_xnext_fulltraj(self, straj, ut, tt):
        part = straj.get_smoothed_estimates()
        M = part.shape[1]
        xp = part
        diff = part[1:] - xp[:-1]
        logp = kalman.lognormpdf_scalar(diff.ravel(), self.Q)
        return numpy.sum(logp) / M
    
    def eval_logp_y_fulltraj(self, straj, yt, tt):
        sest = straj.get_smoothed_estimates()
        M = sest.shape[1]
        yp = numpy.repeat(numpy.asarray(yt, dtype=float).reshape((-1, 1, 1)),
                                 repeats=M, axis=1)
        #return numpy.sum(kalman.lognormpdf_scalar(diff.ravel(), self.R)) / M
        return numpy.sum(bernoulli.logpmf(yp.ravel(), 
                                          numpy.exp(sest.ravel())/(1.0 + numpy.exp(sest.ravel())))) / M


In [11]:
def callback(params, Q, cur_iter):
    print "params = %s" % numpy.exp(params)


def callback_sim(estimator):
    # vals = numpy.empty((num, steps+1))

    plt.figure(1)
    plt.clf()
#    mvals = numpy.empty((steps+1))
#    for k in range(steps+1):
#        #vals[:,k] = numpy.copy(estimator.pt.traj[k].pa.part)
#        mvals[k] = wmean(estimator.pt.traj[k].pa.w,
#                          estimator.pt.traj[k].pa.part)
#        #plt.plot((k,)*num, vals[:,k], 'k.', markersize=0.5)
#    plt.plot(range(steps+1), mvals, 'k-')


    sest_mean = estimator.get_smoothed_mean()
    for k in range(sest_mean.shape[1]):
        plt.plot(range(steps + 1), sest_mean[:, k], 'g-')

    plt.plot(range(steps + 1), x, 'r-')
    plt.plot(range(steps + 1), y, 'bx')
    plt.draw()
    plt.show()


In [12]:
if __name__ == '__main__':
    numpy.random.seed(1)
    steps = 49
    iterations = numpy.asarray(range(500))
    num = numpy.ceil(500 + 4500.0 / (iterations[-1] ** 3) * iterations ** 3).astype(int)
    M = numpy.ceil(50 + 450.0 / (iterations[-1] ** 3) * iterations ** 3).astype(int)
    P0 = 1.0 * numpy.eye(1)
    Q = 1.0 * numpy.eye(1)
    (x, y) = generate_dataset(steps, P0, Q)
    theta0 = numpy.log(numpy.asarray((2.0,)))
    model = Model(P0, Q)
    estimator = param_est.ParamEstimation(model, u=None, y=y)
    callback(theta0, None, -1)
    estimator.maximize(theta0, num, M, fismoother='full', meas_first=True, max_iter=len(iterations),
                       callback=callback)

params = [ 2.]
params = [ 1.96732921]
params = [ 1.94934403]
params = [ 1.95092804]
params = [ 1.83927083]
params = [ 1.82894122]
params = [ 1.79550682]
params = [ 1.70845082]
params = [ 1.62488434]
params = [ 1.53452476]
params = [ 1.50846236]
params = [ 1.52136894]
params = [ 1.5270918]
params = [ 1.48260979]
params = [ 1.4744939]
params = [ 1.4128592]
params = [ 1.42888103]
params = [ 1.43091029]
params = [ 1.37614249]
params = [ 1.386221]
params = [ 1.36045571]
params = [ 1.33475609]
params = [ 1.336462]
params = [ 1.36180927]
params = [ 1.35117558]
params = [ 1.37227483]
params = [ 1.3610797]
params = [ 1.36158648]
params = [ 1.36183459]
params = [ 1.29437593]
params = [ 1.27951723]
params = [ 1.18091937]
params = [ 1.1825177]
params = [ 1.18278728]
params = [ 1.15795362]
params = [ 1.08504069]
params = [ 1.04009552]
params = [ 1.04986278]
params = [ 1.06402855]
params = [ 1.07139392]
params = [ 1.08735707]
params = [ 1.07890764]
params = [ 1.03120681]
params = [ 1.01380827]
param